In [11]:
# -*- coding: UTF-8 -*-

import pandas as pd
import numpy as np

# train_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/train_format1.csv', sep=',')
train_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/train_format1.csv', sep=',')
print("train_format1 loaded successful")
print(train_df.head())

# user_log_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_log_format1.csv')
user_log_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_log_format1.csv', dtype={'time_stamp': str})
print("user_log_format1 loaded successful")
print(user_log_df.head())

# user_info_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_info_format1.csv')
user_info_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_info_format1.csv')
print("user_info_format1 loaded successful")
print(user_info_df.head())

train_with_user_info = pd.merge(train_df, user_info_df)
print(train_with_user_info.head())

user_log_df['month'] = user_log_df['time_stamp'].str[:2]

train_format1 loaded successful
   user_id  merchant_id  label
0    34176         3906      0
1    34176          121      0
2    34176         4356      1
3    34176         2217      0
4   230784         4818      0
user_log_format1 loaded successful
   user_id  item_id  cat_id  seller_id  brand_id time_stamp  action_type
0   328862   323294     833       2882    2661.0       0829            0
1   328862   844400    1271       2882    2661.0       0829            0
2   328862   575153    1271       2882    2661.0       0829            0
3   328862   996875    1271       2882    2661.0       0829            0
4   328862  1086186    1271       1253    1049.0       0829            0
user_info_format1 loaded successful
   user_id  age_range  gender
0   376517        6.0     1.0
1   234512        5.0     0.0
2   344532        5.0     0.0
3   186135        5.0     0.0
4    30230        5.0     0.0
   user_id  merchant_id  label  age_range  gender
0    34176         3906      0        6.0  

In [32]:
user_actions_at_nov = user_log_df.loc[user_log_df['month'] == "11"]
user_actions_not_at_nov = user_log_df.loc[user_log_df['month'] != "11"]

   user_id  item_id  cat_id  seller_id  brand_id time_stamp  action_type month
0   328862   323294     833       2882    2661.0       0829            0    08
1   328862   844400    1271       2882    2661.0       0829            0    08
2   328862   575153    1271       2882    2661.0       0829            0    08
3   328862   996875    1271       2882    2661.0       0829            0    08
4   328862  1086186    1271       1253    1049.0       0829            0    08


In [14]:
print(user_log_df.head())

   user_id  item_id  cat_id  seller_id  brand_id time_stamp  action_type month
0   328862   323294     833       2882    2661.0       0829            0    08
1   328862   844400    1271       2882    2661.0       0829            0    08
2   328862   575153    1271       2882    2661.0       0829            0    08
3   328862   996875    1271       2882    2661.0       0829            0    08
4   328862  1086186    1271       1253    1049.0       0829            0    08


In [29]:
len(user_log_df)

54925330